### Day 7

In [66]:
import copy
import pandas as pd

In [67]:
with open('D7.txt') as f:
    lines = f.readlines()

In [68]:
def get_current_folder(current_path, folders):
    
    tmp_dic = folders

    for folder_name in current_path:

        if folder_name in tmp_dic.keys():
            tmp_dic = tmp_dic[folder_name]
            
    return tmp_dic
    

In [69]:
folders = {}
folders['/'] = {}
current_path = []

for line in lines:
    
    command = line.strip().split(' ')
    
    if command[1] == 'cd':
        
        if command[2] == '..':
            current_path.pop()
        else:
            current_path.append(command[2])
                                
    elif command[0] == 'dir':
        
        current_folder = get_current_folder(current_path, folders)
        current_folder[command[1]] = {}
            
    elif command[0].isnumeric() == True:
        
        current_folder = get_current_folder(current_path, folders)
        current_folder[command[1]] = int(command[0])

In [70]:
def concat_files(fold):
        
    keys_files = [k for k, v in fold.items() if type(v)!=dict]
    fold['files'] = sum([fold[k] for k in keys_files])

    for k in keys_files:
        fold.pop(k)
        
    return fold

In [71]:
folders_cp = copy.deepcopy(folders)

In [77]:
for k,v in folders_cp.items():
    if type(folders_cp[k]) == dict:
        concat_files(folders_cp[k])
        
        for k1,v1 in folders_cp[k].items():
            if type(folders_cp[k][k1]) == dict:
                concat_files(folders_cp[k][k1])
                
                for k2,v2 in folders_cp[k][k1].items():
                    if type(folders_cp[k][k1][k2]) == dict:
                        concat_files(folders_cp[k][k1][k2])
                        
                        for k3,v3 in folders_cp[k][k1][k2].items():
                            if type(folders_cp[k][k1][k2][k3]) == dict:
                                concat_files(folders_cp[k][k1][k2][k3])
                                
                                for k4,v4 in folders_cp[k][k1][k2][k3].items():
                                    if type(folders_cp[k][k1][k2][k3][k4]) == dict:
                                        concat_files(folders_cp[k][k1][k2][k3][k4])

In [73]:
df = pd.json_normalize(folders_cp, sep='_')


In [74]:
df.columns = df.columns.str.replace('_files', '')
folders_cp_flat = df.to_dict(orient='records')[0]

folders_cp_flat_sum = folders_cp_flat.copy()

In [75]:
for k,v in folders_cp_flat_sum.items():
            
    keys = list(folders_cp_flat.keys())
    keys.remove(k)
    
    paths = [s for s in keys if k in s]
    
    if len(paths)>0:
        
        folders_cp_flat_sum[k] = folders_cp_flat[k] + sum([folders_cp_flat[path] for path in paths])
               

In [76]:
total = 0
for v in folders_cp_flat.values():
    if v < 100000:
        total = total + v
    
total

1609610